In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:

import os
from langchain_mcp_adapters.client import MultiServerMCPClient

github_pat = os.getenv("GITHUB_PAT")
slack_bot_token = os.getenv("SLACK_BOT_TOKEN")
slack_team_id = os.getenv("SLACK_TEAM_ID")
slack_channel_ids = os.getenv("SLACK_CHANNEL_IDS")

mcp_client = MultiServerMCPClient({
    "github": {
      "command": "docker",
      "args": [
        "run",
        "-i",
        "--rm",
        "-e",
        "GITHUB_TOOLSETS",
        "-e",
        "GITHUB_PERSONAL_ACCESS_TOKEN",
        "ghcr.io/github/github-mcp-server"
      ],
      "env": {
        "GITHUB_TOOLSETS": "context,pull_requests",
        "GITHUB_PERSONAL_ACCESS_TOKEN": github_pat
      },
      "transport": "stdio"
    },
     "slack": {
      "command": "docker",
      "args": [
        "run",
        "-i",
        "--rm",
        "-e",
        "SLACK_BOT_TOKEN",
        "-e",
        "SLACK_TEAM_ID",
        "-e",
        "SLACK_CHANNEL_IDS",
        "mcp/slack"
      ],
      "env": {
        "SLACK_BOT_TOKEN": slack_bot_token,
        "SLACK_TEAM_ID": slack_team_id,
        "SLACK_CHANNEL_IDS": slack_channel_ids # ,로 구분된 문자열
      },
      "transport": "stdio"
    }
})

In [3]:
tool_list = await mcp_client.get_tools()

In [4]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    model="openai:gpt-4.1",
    tools=tool_list,
    prompt="Use the tools provided to you to answer the user's question"
)

In [5]:
async def process_stream(stream_generator):
    results = []
    try:
        async for chunk in stream_generator:

            key = list(chunk.keys())[0]
            if key == 'agent':
                # Agent 메시지의 내용을 가져옴. 메세지가 비어있는 경우 어떤 도구를 어떻게 호출할지 정보를 가져옴
                content = chunk['agent']['messages'][0].content if chunk['agent']['messages'][0].content != '' else chunk['agent']['messages'][0].additional_kwargs
                print(f"'agent': '{content}'")
            
            elif key == 'tools':
                # 도구 메시지의 내용을 가져옴
                for tool_msg in chunk['tools']['messages']:
                    print(f"'tools': '{tool_msg.content}'")
            
            results.append(chunk)
        return results
    except Exception as e:
        print(f"Error processing stream: {e}")
        return results

In [6]:
from langchain_core.messages import HumanMessage

human_message = """깃헙의 Pull Request를 확인하고 코드 리뷰를 작성해주세요. 
PR의 코드를 리뷰한 후에, 아래 항목을 확인해주세요;
1. 코드가 개선되었는지
2. 예측하지 못한 side effect가 있는지
3. 보안상 문제가 될 수 있는 부분이 없는지

위 내용을 확인해서 PR에 코멘트로 남겨주세요.
그리고 코멘트를 남긴 후에, 슬랙 채널에도 메세지를 전송해서 엔지니어에게 알려주세요
채널: <@C092QEJH6LB>
엔지니어: <@U092VN0DHBP>

PR URL:https://github.com/jasonkang14/sat-reading-client/pull/3 """

stream_generator = agent.astream({"messages": [HumanMessage(human_message)]}, stream_mode="updates")

In [7]:

all_chunks = await process_stream(stream_generator)


if all_chunks:
    final_result = all_chunks[-1]
    print("\nFinal result:", final_result)

'agent': '{'tool_calls': [{'id': 'call_c3kJJAv0icV21zJi1r2E5FbO', 'function': {'arguments': '{"owner":"jasonkang14","repo":"sat-reading-client","pullNumber":3}', 'name': 'get_pull_request'}, 'type': 'function'}], 'refusal': None}'
'tools': '{"id":2626689112,"number":3,"state":"open","locked":false,"title":"commit for the sake of review","body":"회원가입 화면의 코드 구조 개선","created_at":"2025-06-29T05:36:11Z","updated_at":"2025-06-29T05:39:15Z","user":{"login":"jasonkang14","id":45306565,"node_id":"MDQ6VXNlcjQ1MzA2NTY1","avatar_url":"https://avatars.githubusercontent.com/u/45306565?v=4","html_url":"https://github.com/jasonkang14","gravatar_id":"","type":"User","site_admin":false,"url":"https://api.github.com/users/jasonkang14","events_url":"https://api.github.com/users/jasonkang14/events{/privacy}","following_url":"https://api.github.com/users/jasonkang14/following{/other_user}","followers_url":"https://api.github.com/users/jasonkang14/followers","gists_url":"https://api.github.com/users/jasonkan